In [5]:
%%writefile jacobi2d_serial.c
// IT23226814 - Vakesh Ranganathan
// jacobi2d_serial.c - 2D steady state heat diffusion (Laplace) via Jacobi
// Build: gcc -O2 -std=c11 jacobi2d_serial.c -o jacobi2d_serial -lm
// Run:   ./jacobi2d_serial 1024 1024 2000 1e-6
//        ^Nx   ^Ny   ^max_iters ^tolerance
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

static inline size_t idx(int x, int y, int Nx) { return (size_t)y * (size_t)Nx + (size_t)x; }

int main(int argc, char **argv) {
    int Nx = (argc > 1) ? atoi(argv[1]) : 512;
    int Ny = (argc > 2) ? atoi(argv[2]) : 512;
    int max_iters = (argc > 3) ? atoi(argv[3]) : 1000;
    double tol = (argc > 4) ? atof(argv[4]) : 1e-6;

    if (Nx < 3 || Ny < 3 || max_iters <= 0 || tol <= 0.0) {
        fprintf(stderr, "Usage: %s [Nx>=3] [Ny>=3] [max_iters>0] [tol>0]\n", argv[0]);
        return 1;
    }

    size_t N = (size_t)Nx * (size_t)Ny;
    double *u  = (double*)calloc(N, sizeof(double));
    double *un = (double*)calloc(N, sizeof(double));
    if (!u || !un) { fprintf(stderr, "Allocation failed\n"); free(u); free(un); return 1; }

    // Dirichlet BCs: left wall = 1.0, other walls = 0.0
    for (int y = 0; y < Ny; ++y) {
        u[idx(0, y, Nx)] = 1.0;
        un[idx(0, y, Nx)] = 1.0;
    }

    clock_t t0 = clock();
    int iters = 0;
    double last_res = INFINITY;

    for (; iters < max_iters; ++iters) {
        double maxdiff = 0.0;

        // Jacobi interior update (independent doubly-nested loop)
        for (int y = 1; y < Ny - 1; ++y) {
            for (int x = 1; x < Nx - 1; ++x) {
                double v = 0.25 * (u[idx(x-1,y,Nx)] + u[idx(x+1,y,Nx)] +
                                   u[idx(x,y-1,Nx)] + u[idx(x,y+1,Nx)]);
                double d = fabs(v - u[idx(x,y,Nx)]);
                if (d > maxdiff) maxdiff = d;
                un[idx(x,y,Nx)] = v;
            }
        }

        // Enforce boundary (separate loop)
        for (int y = 0; y < Ny; ++y) un[idx(0, y, Nx)] = 1.0;

        // Swap grids
        double *tmp = u; u = un; un = tmp;

        last_res = maxdiff;
        if (maxdiff < tol) break;
    }
    clock_t t1 = clock();

    // Simple checksum (helps verify & prevents dead-code elimination)
    double checksum = 0.0;
    for (size_t i = 0; i < N; ++i) checksum += u[i];

    double secs = (double)(t1 - t0) / (double)CLOCKS_PER_SEC;
    printf("Nx=%d Ny=%d iters=%d time=%.3f s residual=%.3e checksum=%.6f\n",
           Nx, Ny, iters, secs, last_res, checksum);

    free(u); free(un);
    return 0;
}


Overwriting jacobi2d_serial.c


In [6]:
! gcc jacobi2d_serial.c -o jacobi2d_serial

In [7]:
! ./jacobi2d_serial

Nx=512 Ny=512 iters=1000 time=5.340 s residual=2.419e-04 checksum=9058.573482


In [1]:
%%writefile jacobi2d_cuda.cu
// IT23226814 - Vakesh Ranganathan
// jacobi2d_cuda.cu - CUDA implementation of Jacobi Heat Diffusion
// Build: nvcc -O2 -arch=sm_75 jacobi2d_cuda.cu -o jacobi2d_cuda
// Run:   ./jacobi2d_cuda <Nx> <Ny> <Iters> <BlockX> <BlockY>

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

// Error checking macro
#define cudaCheckError(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true) {
   if (code != cudaSuccess) {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

// --- CUDA KERNEL ---
// Each thread calculates one cell (x,y)
__global__ void jacobi_kernel(double *u, double *un, int Nx, int Ny) {
    // Calculate global thread ID
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    // Boundary check: Only update interior points (1 to Nx-2, 1 to Ny-2)
    if (x > 0 && x < Nx - 1 && y > 0 && y < Ny - 1) {
        int idx = y * Nx + x;
        // The stencil operation
        un[idx] = 0.25 * (u[idx - 1] + u[idx + 1] + u[idx - Nx] + u[idx + Nx]);
    }
}

int main(int argc, char **argv) {
    // 1. Parameters
    int Nx = (argc > 1) ? atoi(argv[1]) : 2048;
    int Ny = (argc > 2) ? atoi(argv[2]) : 2048;
    int max_iters = (argc > 3) ? atoi(argv[3]) : 1000;
    // For CUDA benchmarking, we control block size via CLI
    int BLOCK_X = (argc > 4) ? atoi(argv[4]) : 16;
    int BLOCK_Y = (argc > 5) ? atoi(argv[5]) : 16;

    size_t N = (size_t)Nx * (size_t)Ny;
    size_t size_bytes = N * sizeof(double);

    printf("CUDA Configuration: Grid %dx%d, Block Size %dx%d\n", Nx, Ny, BLOCK_X, BLOCK_Y);

    // 2. Host Memory Allocation
    double *h_u = (double*)calloc(N, sizeof(double));
    double *h_un = (double*)calloc(N, sizeof(double));

    // Initialize Boundary Conditions (Left Wall = 1.0)
    for (int y = 0; y < Ny; ++y) {
        h_u[y * Nx] = 1.0;
        h_un[y * Nx] = 1.0; // Ensure 'un' also has BCs set
    }

    // 3. Device Memory Allocation
    double *d_u, *d_un;
    cudaCheckError(cudaMalloc((void**)&d_u, size_bytes));
    cudaCheckError(cudaMalloc((void**)&d_un, size_bytes));

    // Copy Host -> Device
    cudaCheckError(cudaMemcpy(d_u, h_u, size_bytes, cudaMemcpyHostToDevice));
    cudaCheckError(cudaMemcpy(d_un, h_un, size_bytes, cudaMemcpyHostToDevice));

    // 4. Kernel Configuration
    dim3 threadsPerBlock(BLOCK_X, BLOCK_Y);
    dim3 blocksPerGrid((Nx + BLOCK_X - 1) / BLOCK_X, (Ny + BLOCK_Y - 1) / BLOCK_Y);

    // Create CUDA Events for precise timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // --- MAIN LOOP ---
    cudaEventRecord(start);

    for (int i = 0; i < max_iters; ++i) {
        // Launch Kernel
        jacobi_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_u, d_un, Nx, Ny);

        // Swap Pointers on Device (No data movement, just swap addresses)
        double *tmp = d_u; d_u = d_un; d_un = tmp;
    }
    cudaCheckError(cudaGetLastError()); // Check for launch errors

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    // 5. Retrieve Results
    // Copy final grid back to host (d_u is the latest because of swap)
    cudaCheckError(cudaMemcpy(h_u, d_u, size_bytes, cudaMemcpyDeviceToHost));

    // 6. Checksum Verification
    double checksum = 0.0;
    for (size_t i = 0; i < N; ++i) checksum += h_u[i];

    printf("Result: Time=%.3f s, Checksum=%.6f\n", milliseconds / 1000.0, checksum);

    // Cleanup
    cudaFree(d_u); cudaFree(d_un);
    free(h_u); free(h_un);
    cudaEventDestroy(start); cudaEventDestroy(stop);

    return 0;
}

Writing jacobi2d_cuda.cu


In [2]:
!nvcc -O2 -arch=sm_75 jacobi2d_cuda.cu -o jacobi2d_cuda

In [10]:
!./jacobi2d_cuda 1024 1024 1000 16 16

CUDA Configuration: Grid 1024x1024, Block Size 16x16
Result: Time=0.113 s, Checksum=18452.713976


In [11]:
!./jacobi2d_cuda 1024 1024 1000 8 8

CUDA Configuration: Grid 1024x1024, Block Size 8x8
Result: Time=0.112 s, Checksum=18452.713976


In [12]:
!./jacobi2d_cuda 1024 1024 1000 32 32

CUDA Configuration: Grid 1024x1024, Block Size 32x32
Result: Time=0.153 s, Checksum=18452.713976
